In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

CHECKERBOARD = (14, 10)
square_size_mm = 6
subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_PRINCIPAL_POINT
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = (
    np.mgrid[0 : CHECKERBOARD[0], 0 : CHECKERBOARD[1]].T.reshape(-1, 2) * square_size_mm
)
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)

In [ ]:
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

In [ ]:
_ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv2.findChessboardCorners(
    gray,
    CHECKERBOARD,
    cv2.CALIB_CB_ADAPTIVE_THRESH
    + cv2.CALIB_RATIONAL_MODEL
    + cv2.CALIB_FIX_PRINCIPAL_POINT,
    # + cv2.CALIB_CB_FAST_CHECK
    # + cv2.CALIB_CB_NORMALIZE_IMAGE,
)
# If found, add object points, image points (after refining them)
if ret == True:
    objpoints.append(objp)
    cv2.cornerSubPix(gray, corners, (3, 3), (-1, -1), subpix_criteria)
    imgpoints.append(corners)
    cv2.drawChessboardCorners(frame, CHECKERBOARD, corners, ret)
plt.imshow(frame)
plt.show()

If detected output is undesirable, pop that set of obj/img points.


In [ ]:
objpoints.pop()
imgpoints.pop()

In [ ]:
cap.release()

In [ ]:
N_OK = len(objpoints)
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, K, D, _, _ = cv2.calibrateCamera(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    None,
    None,
    flags=calibration_flags,
    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6),
)
print("Found " + str(N_OK) + " valid images for calibration")
# print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
D
print(rms)